In [1]:
!pip install faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 63.5 MB/s eta 0:00:00


In [2]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.0/331.0 kB 6.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from pypdf import PdfReader

# Define the path to your PDF file
path = "/content/sample_data/Foundations_of_Machine_Learning.pdf"

#extract text from pdf
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() or "" # Use or "" to handle pages with no extractable text
    return text

pdf_text = extract_text_from_pdf(path)

In [4]:
pdf_text

'Foundations of Machine LearningAdaptive Computation and Machine Learning\nThomas Dietterich, Editor\nChristopher Bishop, David Heckerman, Michael Jordan, and Michael Kearns,\nAssociate Editors\nA complete list of books published in The Adaptive Computations and Machine\nLearning series appears at the back of this book.Foundations of Machine Learning\nMehryar Mohri, Afshin Rostamizadeh, and Ameet Talwalkar\nThe MIT Press\nCambridge, Massachusetts\nLondon, Englandc⃝ 2012 Massachusetts Institute of Technology\nAll rights reserved. No part of this book may be reproduced in any form by any\nelectronic or mechanical means (including photocopying, recording, or information\nstorage and retrieval) without permission in writing from the publisher.\nMIT Press books may be purchased at special quantity discounts for business or\nsales promotional use. For information, please email special\nsales@mitpress.mit.edu\nor write to Special Sales Department, The MIT Press, 55 Hayward Street, Cam-\nbridg

In [5]:
#chunk paragraps to sentences using nltk
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

sentences = sent_tokenize(pdf_text)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [6]:
# Convert text to vectors / Generate embedding
# Load embedding model
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = model.encode(sentences)
print(embeddings.shape)   # (4(rows), 384)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(5815, 384)


In [7]:
#create vector database using faiss
import faiss

dimension = embeddings.shape[1]
# Create index
index = faiss.IndexFlatL2(dimension)

# Add vectors
index.add(np.array(embeddings))

print("Total vectors stored:", index.ntotal)

Total vectors stored: 5815


In [8]:
# Perform similar search

query = "What is a machine learning?"
query_vector = model.encode([query])

# Search top 10 similar results
k = 10
distances, indices = index.search(np.array(query_vector), k)

print("Most similar documents:")
for idx in indices[0]:
    print(sentences[idx])

Most similar documents:
Machine learning is a relatively recent ﬁeld and yet probably one of the most
active ones in computer science.
Since the success of a learning algorithm depends on the data used, machine
learning is inherently related to data analysis and statistics.
It covers fundamental modern topics in
machine learning while providing the theoretical basis and conceptual tools needed
for the discussion and justiﬁcation of algorithms.
The main practical objectives of machine learning consist of generating accurate
predictions for unseen items and of designing eﬃcient and robust algorithms to
produce these predictions, even for large-scale problems.
Finally, we thank the MIT Press publication team for their help and support in
the development of this text.1 Introduction
Machine learning can be broadly deﬁned as computational methods using experience
to improve performance or to make accurate predictions.
Machine learning consists of designing eﬃcient and accurate prediction alg

In [9]:
query2 = "What is analysis"
query_vector2 = model.encode([query2])

# Search top 10 similar results
k = 10
distances, indices = index.search(np.array(query_vector2), k)

print("Most similar documents:")
for idx in indices[0]:
    print(sentences[idx])

Most similar documents:
Such an algorithm-dependent analysis could have the beneﬁt
of a more informative guarantee.
Visualization: to visualize the data for exploratory analysis by mapping the input
data into two- or three-dimensional spaces.
Here,experience refers to
the past information available to the learner, which typically takes the form of
electronic data collected and made available for analysis.
A ﬁner analysis can be used in fact to eliminate that factor.
Since the success of a learning algorithm depends on the data used, machine
learning is inherently related to data analysis and statistics.
Clustering is often per-
formed to analyze very large data sets.
For example, in the context of social net-
work analysis, clustering algorithms attempt to identify “communities” within large
groups of people.
More generally, learning
techniques are data-driven methods combining fundamental concepts in computer
science with ideas from statistics, probability and optimization.
Regression